In [15]:
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tqdm import tqdm



In [5]:
train_data_path = '/Users/parthsdambhare/Documents/Enc/Movie classifier/Genre Classification Dataset/train_data.txt'
test_data_path = '/Users/parthsdambhare/Documents/Enc/Movie classifier/Genre Classification Dataset/test_data_solution.txt'

# Load train and test data
movies_train = []
with open('/Users/parthsdambhare/Documents/Enc/Movie classifier/Genre Classification Dataset/train_data.txt', 'r') as file:
    for line in file:
        parts = line.strip().split(':::') 
        title = parts[1]
        plot = parts[3]
        genre = parts[2] 
        movies_train.append([title , plot, genre])
        
movies_test = []
with open('/Users/parthsdambhare/Documents/Enc/Movie classifier/Genre Classification Dataset/test_data_solution.txt', 'r') as file:
    for line in file:
        parts = line.strip().split(':::') 
        title = parts[1]
        plot = parts[3]
        genre = parts[2] 
        movies_test.append([title, plot, genre])        

train_df = pd.DataFrame(movies_train, columns=['Title', 'Plot', 'Genre'])
print(train_df.head())

test_df = pd.DataFrame(movies_test, columns=['Title', 'Plot', 'Genre'])
print(test_df.head())

                                Title  \
0       Oscar et la dame rose (2009)    
1                       Cupid (1997)    
2   Young, Wild and Wonderful (1980)    
3              The Secret Sin (1915)    
4             The Unrecovered (2007)    

                                                Plot       Genre  
0   Listening in to a conversation between his do...      drama   
1   A brother and sister with a past incestuous r...   thriller   
2   As the bus empties the students for their fie...      adult   
3   To help their unemployed father make ends mee...      drama   
4   The film's title refers not only to the un-re...      drama   
                           Title  \
0          Edgar's Lunch (1998)    
1      La guerra de papá (1977)    
2   Off the Beaten Track (2010)    
3        Meu Amigo Hindu (2015)    
4             Er nu zhai (1955)    

                                                Plot          Genre  
0   L.R. Brane loves his life - his car, his apar...      thrill

In [9]:
'''# Encode genres into numerical labels
label_encoder = LabelEncoder()
df_train['Genre'] = label_encoder.fit_transform(df_train['Genre'])
df_test['Genre'] = label_encoder.transform(df_test['Genre'])

# Apply TF-IDF Vectorizer to the plot summaries
tfidf = TfidfVectorizer(max_features=5000, stop_words='english')

X_train = tfidf.fit_transform(df_train['Plot']).toarray()
X_test = tfidf.transform(df_test['Plot']).toarray()

y_train = df_train['Genre'].values
y_test = df_test['Genre'].values'''


In [20]:
# Encode the genres into numerical labels
label_encoder = LabelEncoder()
train_df['Genre'] = label_encoder.fit_transform(train_df['Genre'])
test_df['Genre'] = label_encoder.transform(test_df['Genre'])

# Train-validation split
train_texts, val_texts, train_labels, val_labels = train_test_split(
    train_df['Plot'], train_df['Genre'], test_size=0.2, random_state=42
)
test_texts = test_df['Plot']
test_labels = test_df['Genre']

In [21]:
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
# Load BERT tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

# Tokenize function
def tokenize_data(texts, max_length=128):
    return tokenizer(
        texts.tolist(), 
        padding=True, 
        truncation=True, 
        return_tensors="pt", 
        max_length=max_length
    )

# Tokenize train, validation, and test data
train_encodings = tokenize_data(train_texts)
val_encodings = tokenize_data(val_texts)
test_encodings = tokenize_data(test_texts)


In [22]:
class MovieGenreDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

# Create Dataset objects
train_dataset = MovieGenreDataset(train_encodings, train_labels.values)
val_dataset = MovieGenreDataset(val_encodings, val_labels.values)
test_dataset = MovieGenreDataset(test_encodings, test_labels.values)


In [23]:
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)


In [12]:
# Define the model
import torch
num_classes = len(label_encoder.classes_)
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=num_classes)

# Move the model to MPS (Apple Silicon)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
model.to(device)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
 

In [17]:
# Optimizer and loss function
#optimizer = AdamW(model.parameters(), lr=5e-5)
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)


In [18]:


# Training loop
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        # Move batch to the device
        batch = {key: val.to(device) for key, val in batch.items()}
        
        # Forward pass
        outputs = model(**batch)
        loss = outputs.loss
        
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch+1}/{num_epochs}, Training loss: {avg_train_loss:.4f}")

    # Validation
    model.eval()
    total_val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            batch = {key: val.to(device) for key, val in batch.items()}
            outputs = model(**batch)
            loss = outputs.loss
            total_val_loss += loss.item()

            # Calculate accuracy
            logits = outputs.logits
            predictions = torch.argmax(logits, dim=-1)
            correct += (predictions == batch['labels']).sum().item()
            total += len(batch['labels'])

    avg_val_loss = total_val_loss / len(val_loader)
    accuracy = correct / total
    print(f"Validation loss: {avg_val_loss:.4f}, Validation accuracy: {ac curacy:.4f}")


  0%|          | 0/2711 [00:00<?, ?it/s]/var/folders/mc/gvm9y0s93y50g3gglym7mpw40000gn/T/ipykernel_5160/360674222.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 2711/2711 [40:34<00:00,  1.11it/s] 


Epoch 1/3, Training loss: 1.3705
Validation loss: 1.2300, Validation accuracy: 0.6191


100%|██████████| 2711/2711 [2:13:46<00:00,  2.96s/it]    


Epoch 2/3, Training loss: 0.9341
Validation loss: 1.1964, Validation accuracy: 0.6427


100%|██████████| 2711/2711 [3:30:36<00:00,  4.66s/it]     


Epoch 3/3, Training loss: 0.5860
Validation loss: 1.4111, Validation accuracy: 0.6235
